In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
%run lib2 --area_id=5 --bs=32 --num_workers=8 --num_slice=25 --gpu_start=4 --num_gpus=2
pad = True

In [3]:
from v17 import _internal_pred_to_poly_file, _calc_fscore_per_aoi

In [3]:
# valtest
test_csv = pd.read_csv(PATH/DATA_PATH/FMT_VALTEST_FN)
image_ids = test_csv['ImageId'].tolist()
test_y = load_y(image_ids, FMT_VALTEST_MASK_STORE)
x_names = [str(o) + '.png' for o in image_ids]

In [4]:
# actual test
test_csv = pd.read_csv(PATH/DATA_PATH/FMT_TEST_FN)
image_ids = test_csv['ImageId'].tolist()
x_names = [str(o) + '.png' for o in image_ids]

In [5]:
stats = get_rgb_mean_stat(area_id)
aug_tfms = transforms_top_down
for o in aug_tfms: o.tfm_y = TfmType.CLASS
tfms = tfms_from_stats(stats, sz, crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)

In [6]:
model = get_model()

In [7]:
empty = ['empty'] * 2
trn = empty
val = empty
test = x_names
# datasets = ImageData.get_ds(OptionalFilesArrayDataset, trn, val, tfms,
#                             test=test, path=PATH/DATA_PATH/TRN, num_slice=num_slice, pad=pad)
datasets = ImageData.get_ds(OptionalFilesArrayDataset, trn, val, tfms,
                            test=test, path=PATH/DATA_PATH/TEST, num_slice=num_slice, pad=pad)
md = ImageData(str(PATH), datasets, bs, num_workers=num_workers, classes=None)
denorm = md.trn_ds.denorm

In [8]:
learn = get_learn(md, model, load_weight=True)

In [10]:
preds = learn.predict(is_test=True).squeeze()

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
# Or this for test
preds = manual_predict()

In [18]:
%%time
preds_big = np.asarray(list(put_back_parallel(preds, max_workers=1))).squeeze()

CPU times: user 18.5 s, sys: 3.23 s, total: 21.8 s
Wall time: 6.1 s


In [9]:
%%time
preds_big = np.asarray(list(put_back_parallel(preds, max_workers=4))).squeeze()

CPU times: user 25.6 s, sys: 3.64 s, total: 29.2 s
Wall time: 2.59 s


In [13]:
# %debug

In [10]:
preds_big_sigmoid = to_np(F.sigmoid(T(preds_big)))

In [11]:
def find_thresh(preds_big):
    best_fscore, best_thresh = 0, 0
    records, fscores = [], []
    threshs = np.linspace(0.4, 0.9, 6)
    for thresh in threshs:
        _internal_pred_to_poly_file(
                area_id,
                preds_big,
                min_th=MIN_POLYGON_AREA,
                thresh=thresh,
                debug=False)
        evaluate_record = _calc_fscore_per_aoi(area_id)
        evaluate_record['min_area_th'] = MIN_POLYGON_AREA
        evaluate_record['area_id'] = area_id
        evaluate_record['thresh'] = thresh
        fscore = evaluate_record['fscore']
        records.append(evaluate_record)
        fscores.append(fscore)
        print(evaluate_record)
        if fscore > best_fscore:
            best_fscore = fscore
            best_record = evaluate_record
    return best_thresh, fscores, records

In [16]:
best_thresh, fscores, records = find_thresh(preds_big_sigmoid)

{'overall_fscore': 0.166726, 'tp': 1411, 'fp': 8069, 'fn': 6035, 'precision': 0.14884, 'recall': 0.189498, 'fscore': 0.166726, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.4}
{'overall_fscore': 0.183107, 'tp': 1537, 'fp': 7805, 'fn': 5909, 'precision': 0.164526, 'recall': 0.20642, 'fscore': 0.183107, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.5}
{'overall_fscore': 0.198305, 'tp': 1661, 'fp': 7645, 'fn': 5785, 'precision': 0.178487, 'recall': 0.223073, 'fscore': 0.198305, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.6000000000000001}
{'overall_fscore': 0.202854, 'tp': 1720, 'fp': 7792, 'fn': 5726, 'precision': 0.180824, 'recall': 0.230997, 'fscore': 0.202854, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.7000000000000001}
{'overall_fscore': 0.062553, 'tp': 512, 'fp': 8412, 'fn': 6934, 'precision': 0.057373, 'recall': 0.068762, 'fscore': 0.062553, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.8}
{'overall_fscore': 0.022515, 'tp': 123, 'fp': 3357, 'fn': 7323, 'precision': 0.035345, 'recal

In [12]:
best_thresh = 0.7

In [13]:
records, fscores = [], []
for th in range(30, 150, 30):
    _internal_pred_to_poly_file(area_id, preds_big_sigmoid, min_th=th, thresh=best_thresh, debug=False)
    evaluate_record = _calc_fscore_per_aoi(area_id)
    evaluate_record['min_area_th'] = th
    evaluate_record['area_id'] = area_id
    evaluate_record['thresh'] = best_thresh
    fscore = evaluate_record['fscore']
    records.append(evaluate_record)
    fscores.append(fscore)
    print(evaluate_record)
    
best_record = records[np.argmax(fscore)]
pd.DataFrame(best_record).to_csv(
    str(PATH/DATA_PATH/FMT_VALMODEL_EVALTHHIST),
    index=False)

{'overall_fscore': 0.202854, 'tp': 1720, 'fp': 7792, 'fn': 5726, 'precision': 0.180824, 'recall': 0.230997, 'fscore': 0.202854, 'min_area_th': 30, 'area_id': 5, 'thresh': 0.7}
{'overall_fscore': 0.227848, 'tp': 1710, 'fp': 5854, 'fn': 5736, 'precision': 0.226071, 'recall': 0.229654, 'fscore': 0.227848, 'min_area_th': 60, 'area_id': 5, 'thresh': 0.7}
{'overall_fscore': 0.240943, 'tp': 1686, 'fp': 4863, 'fn': 5760, 'precision': 0.257444, 'recall': 0.22643, 'fscore': 0.240943, 'min_area_th': 90, 'area_id': 5, 'thresh': 0.7}
{'overall_fscore': 0.247865, 'tp': 1654, 'fp': 4246, 'fn': 5792, 'precision': 0.280339, 'recall': 0.222133, 'fscore': 0.247865, 'min_area_th': 120, 'area_id': 5, 'thresh': 0.7}


ValueError: If using all scalar values, you must pass an index